In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [19]:
data = pd.read_csv('04.reduced_telco_data.csv')
data.head()

,SeniorCitizen,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,Contract_Month-to-Month,Contract_One year,Contract_Two year,...,DeviceProtection_Yes,TechSupport_No,TechSupport_Unknown,TechSupport_Yes,StreamingTV_No,StreamingTV_Unknown,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Unknown,StreamingMovies_Yes
0,0,-0.244200,1,1,0.021083,0.001391,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1
1,1,-0.306748,1,0,0.053733,0.002613,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,0,-0.744587,1,0,0.004393,0.000401,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0
3,0,1.069316,1,1,0.020597,0.002886,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
4,0,-0.619490,1,0,0.041444,0.001535,0,1,0,0,...,1,1,0,0,1,0,0,1,0,0


In [20]:
y = data.iloc[:,6].values
y

array([0, 0, 0, ..., 1, 1, 1], shape=(56000,))

In [21]:
tmp = data.drop(data.columns[6], axis=1)
X = tmp.iloc[:,:].values
X

array([[ 0.        , -0.2441996 ,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.        , -0.30674798,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        , -0.74458664,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.6940261 ,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        , -0.30674798,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        , -1.24497368,  1.        , ...,  1.        ,
         0.        ,  0.        ]], shape=(56000, 46))

In [22]:
X_train,X_test,y_train,y_test = train_test_split(
    X,y,
    test_size=0.2,
    random_state= 42,
    stratify=y,
)

In [23]:
# 1. Initialize the Logistic Regression model
# The C parameter controls regularization (smaller C means stronger regularization)
model = LogisticRegression(random_state=42, solver='liblinear', C=1.0)

# 2. Train the model using the .fit() method
# The model learns from the scaled training features and the corresponding labels
model.fit(X_train, y_train) 

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


In [24]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

accuracy

0.7758035714285715

In [25]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
auc

0.7957728776349003

In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.88      0.66      0.76      5952
           1       0.70      0.90      0.79      5248

    accuracy                           0.78     11200
   macro avg       0.79      0.78      0.77     11200
weighted avg       0.80      0.78      0.77     11200



In [27]:
# saving the model
import joblib
import os

# Define the filename and path
model_filename = 'logistic_regression_model.joblib'

# Save the model
joblib.dump(model, model_filename)

['logistic_regression_model.joblib']

In [28]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

# 1. Load the model from the file
loaded_model = joblib.load(model_filename)

In [29]:
def tenure_z_score_calculation (value: float) -> float:
    import pandas as pd
    import pickle
    with open('tenure_zscore_scaler.pkl', 'rb') as f:
        tenure_scaler = pickle.load(f)
    tenure_df = pd.DataFrame({'tenure': [value]})
    final = tenure_scaler.transform(tenure_df)[0][0]
    return final



In [30]:
def MonthlyCharges_TotalCharges_minmaxscaler_calculation (MonthlyCharges_value: float, TotalCharges_value:float) -> float:
    import pandas as pd
    import pickle
    with open('charges_minmax_scaler.pkl', 'rb') as f:
        charges_scaler = pickle.load(f)
    MonthlyCharges_TotalCharges_df = pd.DataFrame({'MonthlyCharges': [68.61],'TotalCharges': [205.83]})
    MonthlyCharges_final = charges_scaler.transform(MonthlyCharges_TotalCharges_df)[0][0]
    TotalCharges_final = charges_scaler.transform(MonthlyCharges_TotalCharges_df)[0][1]
    return MonthlyCharges_final,TotalCharges_final



In [31]:
def predict_new_customer_churn(customer_data, model, threshold=0.5):
    import numpy as np
    # Convert input to numpy array
    data_array = np.array(customer_data, dtype=np.float32).reshape(1, -1)

    # Get churn probability (probability of class 1)
    probability = model.predict_proba(data_array)[:, 1][0]

    # Apply threshold
    prediction = 1 if probability >= threshold else 0

    return prediction, probability

In [32]:
SeniorCitizen=0
tenure=3
PhoneService=1
PaperlessBilling=0
MonthlyCharges=68.61
TotalCharges=205.83
Contract_Month_to_Month=1 
Contract_One_year=0
Contract_Two_year=0 
PaymentMethod_Bank_transfer_automatic=0
PaymentMethod_CreditP_card_automatic=0
PaymentMethod_Electronic_check=0
PaymentMethod_Mailed_check=1
PaymentMethod_Unknown=0
gender_Female=1
gender_Unknown=0  
Partner_No=1
Partner_Unknown=0
Partner_Yes=0
Dependents_No=0
Dependents_Unknown=0
Dependents_Yes=1
MultipleLines_No=0
MultipleLines_Unknown=0
MultipleLines_Yes=1
InternetService_DSL=0
InternetService_Fiber_optic=0
InternetService_No=1
OnlineSecurity_No=1
OnlineSecurity_Unknown=0
OnlineSecurity_Yes=0
OnlineBackup_No=1
OnlineBackup_Unknown=0
OnlineBackup_Yes=0
DeviceProtection_No=1
DeviceProtection_Unknown=0
DeviceProtection_Yes=0
TechSupport_No=1
TechSupport_Unknown=0
TechSupport_Yes=0
StreamingTV_No=1
StreamingTV_Unknown=0
StreamingTV_Yes=0
StreamingMovies_No=1
StreamingMovies_Unknown=0
StreamingMovies_Yes=0


tenure = tenure_z_score_calculation(tenure)
MonthlyCharges,TotalCharges=MonthlyCharges_TotalCharges_minmaxscaler_calculation(68.61,205.83)



In [33]:
sample_customer_features = [
    SeniorCitizen, tenure, PhoneService, PaperlessBilling, MonthlyCharges, TotalCharges, Contract_Month_to_Month, Contract_One_year, Contract_Two_year, PaymentMethod_Bank_transfer_automatic, PaymentMethod_CreditP_card_automatic, PaymentMethod_Electronic_check, PaymentMethod_Mailed_check, PaymentMethod_Unknown, gender_Female, gender_Unknown, Partner_No, Partner_Unknown, Partner_Yes, Dependents_No, Dependents_Unknown, Dependents_Yes, MultipleLines_No, MultipleLines_Unknown, MultipleLines_Yes, InternetService_DSL, InternetService_Fiber_optic, InternetService_No, OnlineSecurity_No, OnlineSecurity_Unknown, OnlineSecurity_Yes, OnlineBackup_No, OnlineBackup_Unknown, OnlineBackup_Yes, DeviceProtection_No, DeviceProtection_Unknown, DeviceProtection_Yes, TechSupport_No, TechSupport_Unknown, TechSupport_Yes, StreamingTV_No, StreamingTV_Unknown, StreamingTV_Yes, StreamingMovies_No, StreamingMovies_Unknown, StreamingMovies_Yes
]

In [34]:
# Assuming your model is loaded as 'loaded_model'
prediction, probability = predict_new_customer_churn(
    customer_data=sample_customer_features, 
    model=loaded_model, 
    threshold=0.5
)
prediction, probability

(1, np.float64(0.7029611577330708))